<a href="https://colab.research.google.com/github/jyanivaddi/ERA_V1/blob/master/session_19/SAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Connect to GDrive

In [2]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


Clone the repos

In [1]:
!git clone --recurse-submodules "https://github.com/jyanivaddi/ERA_V1.git"
!git -C ERA_V1 pull
#!cd ../
!git clone "https://github.com/jyanivaddi/dl_hub.git"
!git -C dl_hub pull
!git pull

!pip install --quiet "torchinfo" "seaborn" "pytorch-lightning" "torchmetrics" "lightning-bolts" "torchtext" "datasets" "tokenizers" "transformers"

Cloning into 'ERA_V1'...
remote: Enumerating objects: 1927, done.
remote: Counting objects: 100% (520/520), done.
remote: Compressing objects: 100% (314/314), done.
remote: Total 1927 (delta 247), reused 456 (delta 205), pack-reused 1407
Receiving objects: 100% (1927/1927), 236.90 MiB | 28.79 MiB/s, done.
Resolving deltas: 100% (939/939), done.
Updating files: 100% (211/211), done.
Submodule 'session_19/FastSAM' (https://github.com/CASIA-IVA-Lab/FastSAM.git) registered for path 'session_19/FastSAM'
Cloning into '/content/ERA_V1/session_19/FastSAM'...
remote: Enumerating objects: 1171, done.        
remote: Counting objects: 100% (242/242), done.        
remote: Compressing objects: 100% (56/56), done.        
remote: Total 1171 (delta 201), reused 186 (delta 186), pack-reused 929        
Receiving objects: 100% (1171/1171), 72.45 MiB | 21.84 MiB/s, done.
Resolving deltas: 100% (456/456), done.
Submodule path 'session_19/FastSAM': checked out '38654b76b31d3178a8572579968f8eb404936318'
A

In [ ]:
import torch
import torchvision
print("PyTorch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)
print("CUDA is available:", torch.cuda.is_available())
import sys
!{sys.executable} -m pip install opencv-python matplotlib
!{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

!mkdir images
!wget -P images https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/dog.jpg

!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

PyTorch version: 2.1.0+cu118
Torchvision version: 0.16.0+cu118
CUDA is available: False
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-u6gvrhii
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-u6gvrhii
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done
  Created wheel for segment-anything: filename=segment_anything-1.0-py3-none-any.whl size=36586 sha256=502de0516eaed94565321cb875f0e189a46f2265dcf5cc21c8a58254e825fcd1
  Stored in directory: /tmp/pip-ephem-wheel-cache-v4lusd1q/wheels/10/cf/59/9ccb2f0a1bcc81d4fbd0e501680b5d088d690c6cfbc02dc99d
Successfully built segment-anything
--2023-10-19 16:48:00--  https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/dog.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.co

In [5]:
!pip install -r '/content/ERA_V1/session_19/FastSAM/requirements.txt'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.7/611.7 kB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 14.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.7/302.7 kB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 15.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mdit-py-plugins 

Install other dependencies

In [6]:
!pip install git+https://github.com/openai/CLIP.git


  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-ibui7dum
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-ibui7dum
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.4 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369497 sha256=87263fbdd4b422ae7b3c8f9b6b8808f8301bce0aa4feaf6a639eecfbf70bedb6
  Stored in directory: /tmp/pip-ephem-wheel-cache-_ms0vygd/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


In [3]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2

In [19]:
import sys
import os
sys.path.append('/content/ERA_V1/session_19/FastSAM')
from fastsam import FastSAM, FastSAMPrompt

In [29]:
from PIL import Image

def infer_fast_sam(params):
    # load model
    model = FastSAM(params['model_path'])
    device = params['device']
    text_prompt = params['text_prompt']
    input = Image.open(params['input_image'])
    input = input.convert("RGB")
    everything_results = model(input, device=device)
    bboxes = None
    points = None
    point_label = None
    prompt_process = FastSAMPrompt(input, everything_results, device=device)
    ann = prompt_process.text_prompt(text=text_prompt)
    #ann = prompt_process.everything_prompt()
    prompt_process.plot(
        annotations=ann,
        output_path=params['output_dir'],
        bboxes = bboxes,
        points = points,
        point_label = point_label,
        withContours=False,
        better_quality=False,
    )


In [32]:
model_path = '/content/gdrive/MyDrive/trained_models/FastSAM/FastSAM-x.pt'
input_image = '/content/ERA_V1/session_19/FastSAM/images/cat.jpg'
params = {
    'point_prompt': [[0,0]],
    'device': torch.device("cuda") if torch.cuda.is_available() else "cpu",
    'model_path': model_path,
    'text_prompt': 'black dog',
    'input_image': input_image,
    'output_dir': './testimg.png'

}

In [33]:
infer_fast_sam(params)


0: 1024x576 38 objects, 10320.3ms
Speed: 11.2ms preprocess, 10320.3ms inference, 308.9ms postprocess per image at shape (1, 3, 1024, 1024)
